In [2]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [3]:
# define lattice and generate hopping matrix with OBC

In [4]:
from pythtb import *
def Tmatrix(t,mu):

    # set model parameters
    mu=0.0
    t=-1.0
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])

    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [321]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return np.diag(np.exp(phi * Vdiag)).dot(ExpT)

In [6]:
def SVD_stablizer_back(A,U,D,V):
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1
    U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
    return U.dot(U1), D, V
    
def SVD_stablizer_forward(A,U,D,V):
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V1 = LA.svd(A.dot(U.dot(np.diag(D))))
    return U, D, V1.dot(V)

In [7]:
def GF(UL,DL,VL,UR,DR,VR):
    # B(t,0) = UL * DL * VL
    # B(beta,t) = VR * DR * UR Take care of it !!!
    U,D,V = LA.svd(LA.inv(UL.dot(UR))+np.diag(DR).dot(VR.dot(VL)).dot(np.diag(DL)))
    return LA.inv(V.dot(UL)).dot(np.diag(1/D)).dot(LA.inv(UR.dot(U)))

In [8]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpT: the matrix of exp(-dtau * T)
    # Vdiag: Ntau x L matrix with each column being - alpha 
    Ntau, L = phi.shape
    U = np.zeros((Ntau+1,L,L))
    D = np.zeros((Ntau+1,L))
    V = np.zeros((Ntau+1,L,L))

    U[0],D[0],V[0] = LA.svd(np.eye(L))
    for i in range(Ntau):
        U[i+1], D[i+1], V[i+1] = SVD_stablizer_forward(ExpM(phi[i], ExpT, Vdiag),U[i],D[i],V[i])
    return U,D,V

In [757]:
def Update_HS(GFup, GFdn, phi, alpha):
    Ident = np.eye(len(phi))
    for i in range(len(phi)):
        dup = np.exp(-2 * alpha * phi[i]) -1 
        ddn = np.exp(2 * alpha * phi[i]) -1
        Rup = 1 + dup * (1 - GFup[i,i])
        Rdn = 1 + ddn * (1 - GFdn[i,i])
        
        if  np.abs(Rup * Rdn) > np.random.rand():
            
            GFup -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
            GFdn -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn
            phi[i] = -phi[i]
    
    return GFup, GFdn, phi

In [783]:
def Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)
    Ntau = len(phi) 
    ind_revered = np.roll(np.arange(Ntau+1)[::-1], shift=1)
    for n in range(1,Ntau+1):
        i = ind_revered[n]
        i_1 = ind_revered[n-1]
        if np.mod(n,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i_1],Dup[i_1],Uup[i_1],Uup[i],Dup[i],Vup[i])
            GFdn_recomputed = GF(Vdn[i_1],Ddn[i_1],Udn[i_1],Udn[i],Ddn[i],Vdn[i])
            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            print("Error of Green function: ", (GFup_error + GFdn_error)/2)
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1], alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = LA.inv(Bup).dot(GFup).dot(Bup)
        GFdn = LA.inv(Bdn).dot(GFdn).dot(Bdn)

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_back(Bup, Uup[i_1], Dup[i_1], Vup[i_1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_back(Bdn, Udn[i_1], Ddn[i_1], Vdn[i_1])

    return GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi


In [789]:
def Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)

    # reverse the order of U, D, V, and shift B(0,0) to the first one, to make the sweep backward
    
    Ntau = len(phi) 
    for i in range(1,Ntau+1):
        if np.mod(i,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i],Dup[i],Uup[i],Uup[i-1],Dup[i-1],Vup[i-1])
            GFdn_recomputed = GF(Vdn[i],Ddn[i],Udn[i],Udn[i-1],Ddn[i-1],Vdn[i-1])

            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            print("Error of Green function: ", (GFup_error + GFdn_error)/2)
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1], alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = Bup.dot(GFup).dot(LA.inv(Bup))
        GFdn = Bdn.dot(GFdn).dot(LA.inv(Bdn))

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_forward(Bup, Uup[i-1], Dup[i-1], Vup[i-1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_forward(Bdn, Udn[i-1], Ddn[i-1], Vdn[i-1])

    return GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi

In [801]:
L = 4 * 4
Ntau = 2
Nstable = 1
dtau = 0.1
U = 4.0
t = -1.0
mu = 0
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu).real
ExpT = sla.expm(-dtau * T)
Vdiag = np.zeros(L)
Vdiag[:] = alpha

In [803]:
phi = np.random.choice([1, -1], size=(Ntau, L))
#The part of spin up and spin down are not coupled but direct product
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])
# Sweep
GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)
print('-------------------------------------------------------')
#GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)

# Measurement 

Error of Green function:  0.0
Error of Green function:  5.279120985301165e-15
-------------------------------------------------------


In [806]:
GF1 = GF(Vup[0],Dup[0],Uup[0],Uup[1],Dup[1],Vup[1])
GF2 = GF(Vup[1],Dup[1],Uup[1],Uup[0],Dup[0],Vup[0])
print(np.allclose(GF1,GF2))
np.allclose(GFup, GF1)

True


True

In [810]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])
GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)


phi_t = phi.copy()
i= 0
GFup, GFdn = Update_HS_test(GFup, GFdn, phi_t[0], i, alpha)
phi_t[0,i] = -phi_t[0,i]

Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup1 = GF(Vup1[0],Dup1[0],Uup1[0],Uup1[-1],Dup1[-1],Vup1[-1])
GFdn1 = GF(Vdn1[0],Ddn1[0],Udn1[0],Udn1[-1],Ddn1[-1],Vdn1[-1])
print(np.allclose(GFup,GFup1))
print(np.abs(GFup-GFup1).max())

Error of Green function:  0.0
Error of Green function:  2.5164332423388558e-15
0.6352847108801076 2.34851679772067 -0.7294953514930461 2.6967941420581276
False
0.044288243702605985


In [829]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])
GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)

B1 = Uup[1].dot(np.diag(Dup[1])).dot(Vup[1])
B2 = Uup[2].dot(np.diag(Dup[2])).dot(Vup[2])

print(np.allclose(B1, B2.dot(ExpM(phi[0],ExpT,Vdiag))))

Uup1,Dup1,Vup1 = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi,ExpT,-Vdiag)

B3 = Uup1[-1].dot(np.diag(Dup1[-1])).dot(Vup1[-1])
print(np.allclose(B3,B1))

phi_t = phi.copy()
i= 0
GFup, GFdn = Update_HS_test(GFup, GFdn, phi_t[0], i, alpha)
dup = np.exp(-2 * alpha * phi_t[0][i]) -1 

phi_t[0,i] = -phi_t[0,i]
Uup2,Dup2,Vup2 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn2,Ddn2,Vdn2 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
B3 = Uup2[-1].dot(np.diag(Dup2[-1])).dot(Vup2[-1])

delta = np.zeros((len(phi_t[0]),len(phi_t[0])))
delta[i,i] = dup
print(dup)
print(np.allclose((Ident + delta).dot(ExpM(phi[0],ExpT,Vdiag)), ExpM(phi_t[0],ExpT,Vdiag) ))
print(np.allclose(B2.dot((Ident + delta).dot(ExpM(phi[0],ExpT,Vdiag))), B3))
print(np.allclose(ExpM(phi_t[1],ExpT,Vdiag).dot(ExpM(phi_t[0],ExpT,Vdiag)), B3))
print(np.allclose(B2.dot((Ident + delta).dot(ExpM(phi[0],ExpT,Vdiag))), ExpM(phi_t[1],ExpT,Vdiag).dot(ExpM(phi_t[0],ExpT,Vdiag))))

print(np.allclose((Ident + delta).dot(B1), B3))



Error of Green function:  0.0
Error of Green function:  1.438275496964958e-15
True
True
2.3484428972299622 0.6352647204538422 2.6967941420581276 -0.7294953514930461
2.6967941420581276
True
True
True
True
False


In [698]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

In [712]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

phi_t = phi.copy()
GFup, GFdn = Update_HS_test(GFup, GFdn, phi_t[-1], 4, alpha)
phi_t[-1,4] = -phi_t[-1,4]
Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup1 = GF(Vup1[0],Dup1[0],Uup1[0],Uup1[-1],Dup1[-1],Vup1[-1])
GFdn1 = GF(Vdn1[0],Ddn1[0],Udn1[0],Udn1[-1],Ddn1[-1],Vdn1[-1])
print(np.allclose(GFup2,GFup1))
print(np.abs(GFup2-GFup1).max())

True
1.5126788710517758e-15


In [752]:
def Update_HS_test(GFup, GFdn, phi, i, alpha):
    Ident = np.eye(len(phi))
    dup = np.exp(-2 * alpha * phi[i]) -1 
    ddn = np.exp(2 * alpha * phi[i]) -1
    Rup = 1 + dup * (1 - GFup[i,i])
    Rdn = 1 + ddn * (1 - GFdn[i,i])
    print(Rup,Rdn,dup,ddn)
    GFup1 = GFup.copy()
    GFdn1 = GFdn.copy()
    GFup1 -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
    GFdn1 -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn

    return GFup1, GFdn1
def Update_HS(GFup, GFdn, phi, alpha):
    Ident = np.eye(len(phi))
    for i in range(2):
        dup = np.exp(-2 * alpha * phi[i]) -1 
        ddn = np.exp(2 * alpha * phi[i]) -1
        Rup = 1 + dup * (1 - GFup[i,i])
        Rdn = 1 + ddn * (1 - GFdn[i,i])
        print(Rup,Rdn,dup,ddn)
        GFup -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
        GFdn -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn
        phi[i] = -phi[i]
    
    return GFup, GFdn, phi

In [758]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

phi_t = phi.copy()
GFup, GFdn, phi_t[-1] = Update_HS(GFup, GFdn, phi_t[-1], alpha)
Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup1 = GF(Vup1[0],Dup1[0],Uup1[0],Uup1[-1],Dup1[-1],Vup1[-1])
GFdn1 = GF(Vdn1[0],Ddn1[0],Udn1[0],Udn1[-1],Ddn1[-1],Vdn1[-1])
print(np.allclose(GFup,GFup1))
print(np.abs(GFup-GFup1).max())

True
1.5543122344752192e-15


In [ ]:
2.3483602604135703 0.635242366810872 2.6967941420581276 -0.7294953514930461
2.3334606785685095 0.6312119606609723 2.6967941420581276 -0.7294953514930461

In [739]:
phi_t[-1]

array([ 1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1])

In [716]:
def Update_HS_test(GFup, GFdn, phi, i, alpha):
    Ident = np.eye(len(phi))
    dup = np.exp(-2 * alpha * phi[i]) -1 
    ddn = np.exp(2 * alpha * phi[i]) -1
    Rup = 1 + dup * (1 - GFup[i,i])
    Rdn = 1 + ddn * (1 - GFdn[i,i])
    print(Rup,Rdn,dup,ddn)
    GFup1 = GFup.copy()
    GFdn1 = GFdn.copy()
    GFup1 -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
    GFdn1 -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn

    return GFup1, GFdn1

In [747]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

phi_t = phi.copy()
for i in range(2):
    if np.random.rand():
        GFup, GFdn = Update_HS_test(GFup, GFdn, phi_t[-1], i, alpha)
        phi_t[-1,i] = -phi_t[-1,i]

Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup1 = GF(Vup1[0],Dup1[0],Uup1[0],Uup1[-1],Dup1[-1],Vup1[-1])
GFdn1 = GF(Vdn1[0],Ddn1[0],Udn1[0],Udn1[-1],Ddn1[-1],Vdn1[-1])
print(np.allclose(GFup,GFup1))
print(np.abs(GFup-GFup1).max())

2.3483602604135703 0.635242366810872 2.6967941420581276 -0.7294953514930461
2.3334606785685095 0.6312119606609723 2.6967941420581276 -0.7294953514930461
True
1.8967032805461415e-15


In [ ]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return np.diag(np.exp(phi * Vdiag)).dot(ExpT)

In [ ]:
def SVD_stablizer_back(A,U,D,V):
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1
    U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
    return U.dot(U1), D, V
    
def SVD_stablizer_forward(A,U,D,V):
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V1 = LA.svd(A.dot(U.dot(np.diag(D))))
    return U, D, V1.dot(V)

In [431]:
np.allclose(Uup1[-2].dot(np.diag(Dup1[-2])).dot(Vup1[-2]), Uup[-2].dot(np.diag(Dup[-2])).dot(Vup[-2]))

True

In [430]:
print(phi_t)
print(phi)

[[ 1 -1  1 -1  1  1  1  1  1  1 -1 -1 -1  1  1  1]
 [ 1  1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1]]
[[ 1 -1  1 -1  1  1  1  1  1  1 -1 -1 -1  1  1  1]
 [-1  1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1]]


In [550]:
phi_t = phi.copy()
print(phi_t)
for i in range(16):
    if np.random.rand() < 0.5:
        GFup, GFdn = Update_HS_test(GFup, GFdn, phi_t[-1], i, alpha)
        phi_t[-1][i] = -phi_t[-1][i]

[[ 1 -1  1 -1  1  1  1  1  1  1 -1 -1 -1  1  1  1]
 [-1  1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1]]
2.6967941420581276 -0.7294953514930461 3.5182946563826096 0.9517150593686718
-0.7294953514930461 2.6967941420581276 0.9989265861229224 3.692825951925343
2.6967941420581276 -0.7294953514930461 3.6957431456884384 0.9997157005964352
-0.7294953514930461 2.6967941420581276 0.9855585786788038 3.6434071803149357
-0.7294953514930461 2.6967941420581276 0.9961884753056004 3.6827037198955614
-0.7294953514930461 2.6967941420581276 0.9505547450857379 3.514005213338514
2.6967941420581276 -0.7294953514930461 3.642670294119625 0.9853592475375514
-0.7294953514930461 2.6967941420581276 0.8506052057389052 3.144512341779731
2.6967941420581276 -0.7294953514930461 3.505824390924964 0.948341794594263
2.6967941420581276 -0.7294953514930461 3.6431250330512404 0.9854822565324105
2.6967941420581276 -0.7294953514930461 3.681695444940936 0.995915732243401
2.6967941420581276 -0.7294953514930461 3.5147962860007826 

In [551]:
Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup1 = GF(Vup1[0],Dup1[0],Uup1[0],Uup1[-1],Dup1[-1],Vup1[-1])
GFdn1 = GF(Vdn1[0],Ddn1[0],Udn1[0],Udn1[-1],Ddn1[-1],Vdn1[-1])

In [552]:
print(np.allclose(GFup,GFup1))
print(np.abs(GFup - GFup1).max())

False
0.49831819595021043


In [89]:
GFup, GFdn, _  = Update_HS(GFup, GFdn, phi[-1],alpha)

[-1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1  1]


In [142]:
def update_gr_lowrank(gr, i,  phi, alpha):
    gr = np.array(gr)
    L = gr[0].shape[0]
    ratio = np.zeros(2, dtype=np.float64)
    gamma = np.zeros(2, dtype=np.float64)
    bb = [+1, - 1]
    for s in np.arange(2):
        gamma[s] = np.exp(-2* phi[i] * alpha * bb[s]) -1
        ratio[s] = 1 + (1 - gr[s, i, i]) * gamma[s]

    gr_old = gr.copy()  
    for s in np.arange(2):
        for j in np.arange(L):
            for k in np.arange(L):
                if (i == k):
                    gr[s, j, k] = gr_old[s, j, k] \
                        - ((1.0 - gr_old[s, i, k]) * gamma[s] * gr_old[s, j, i]) / ratio[s]
                else:
                    gr[s, j, k] = gr_old[s, j, k]  \
                        + (gr_old[s, i, k] * gamma[s] * gr_old[s, j, i]) / ratio[s]
    return gr

In [718]:
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])

In [719]:
phi_t = phi.copy()
i = 0
Ident = np.eye(len(phi_t[-1]))
dup = np.exp(-2 * alpha * phi_t[-1][i]) -1 
ddn = np.exp(2 * alpha * phi_t[-1][i]) -1
Rup = 1 + dup * (1 - GFup[i,i])
Rdn = 1 + ddn * (1 - GFdn[i,i])
GFup1 = GFup.copy()
GFdn1 = GFdn.copy()

GFup1 -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
GFdn1 -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn

In [720]:
print(Rup,Rdn, dup, ddn)

2.3483602604135703 0.635242366810872 2.6967941420581276 -0.7294953514930461


In [708]:
delta = np.zeros((len(phi_t[-1]),len(phi_t[-1])))
delta[i,i] = dup

In [709]:
phi_t[-1,i] = -phi_t[-1,i]
Uup1,Dup1,Vup1 = Stable_Bmatrix(phi_t,ExpT,Vdiag)
Udn1,Ddn1,Vdn1 = Stable_Bmatrix(phi_t,ExpT,-Vdiag)
GFup2 = GF(Vup1[-1],Dup1[-1],Uup1[-1],Uup1[0],Dup1[0],Vup1[0])
GFdn2 = GF(Vdn1[-1],Ddn1[-1],Udn1[-1],Udn1[0],Ddn1[0],Vdn1[0])

In [710]:
print(np.allclose(GFup1,GFup2))
print(np.max(np.abs(GFup1-GFup2)))

True
1.1726730697603216e-15


In [692]:
np.allclose(ExpM(phi_t[-1],ExpT,Vdiag), (Ident + delta).dot(ExpM(phi[-1],ExpT,Vdiag)))

True

In [693]:
np.allclose(ExpM(phi_t[-1],np.eye(16),Vdiag), (Ident + delta).dot(ExpM(phi[-1],np.eye(16),Vdiag)))

True

In [677]:
np.diag(ExpM(phi_t[-1],np.eye(16),Vdiag))/np.diag(ExpM(phi[-1],np.eye(16),Vdiag))

array([1.        , 1.        , 1.        , 1.        , 3.69679414,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        ])

In [523]:
phi_t = phi.copy()
GFup3, GFdn3 = Update_HS_test(GFup, GFdn, phi_t[-1], 0, alpha)
print(dup,ddn,Rup,Rdn)
np.allclose(GFup1,GFup3)

2.6967941420581276 -0.7294953514930461 2.329807485420126 0.630223754932441
2.6967941420581276 -0.7294953514930461 2.329807485420126 0.630223754932441


True